In [1]:
LANGHCHAIN_TRACINB_V2 = "true"
LANGCHAIN_API_KEY = "ls__ef7c69da49cf48719e4d7c4b33a3debd"
import os
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [2]:
import json

with open("train.json", "r") as file:
    dataset = json.load(file)
file.close()

In [7]:
from langchain_community.llms import Ollama

llm = Ollama(model='qwen')

In [6]:
llm.invoke("What are rhetorical roles in legal domain?")

" In the legal domain, rhetorical roles refer to the various positions that individuals may take when participating in a legal argument or debate. These roles involve persuading others of the validity and importance of specific points of view or arguments. Some common rhetorical roles in the legal domain include:\n\n1. Advocate: An advocate is an individual who represents a party in a legal proceeding, arguing on their behalf to present the strongest possible case. They use rhetoric to persuade the judge, jury, or other decision-maker of the merits of their client's position.\n\n2. Prosecutor: A prosecutor is responsible for presenting the case against a defendant in a criminal trial. Their role involves using rhetoric to convince the jury or judge that the defendant is guilty beyond a reasonable doubt and that they should be convicted of the charges.\n\n3. Defense Attorney: A defense attorney represents the interests of the accused in a criminal case, working to prove their innocence 

In [8]:
label_abbrv = {
    'preamble': 'PREAMBLE',
    'facts': 'FAC',
    'ruling by lower court': 'RLC',
    'issues': 'ISSUE',
    'argument by petitioner': 'ARG_PETITIONER',
    'argument by respondent': 'ARG_RESPONDENT',
    'analysis': 'ANALYSIS',
    'statute ': 'STA',
    'precedent relied': 'PRE_RELIED',
    'precedent not relied': 'PRE_NOT_RELIED',
    'ratio of the decision': 'Ratio',
    'ruling by present court': 'RPC',
    'none': 'NONE'
}

abbrv_label = {v: k for k, v in label_abbrv.items()}
abbrv_label

{'PREAMBLE': 'preamble',
 'FAC': 'facts',
 'RLC': 'ruling by lower court',
 'ISSUE': 'issues',
 'ARG_PETITIONER': 'argument by petitioner',
 'ARG_RESPONDENT': 'argument by respondent',
 'ANALYSIS': 'analysis',
 'STA': 'statute ',
 'PRE_RELIED': 'precedent relied',
 'PRE_NOT_RELIED': 'precedent not relied',
 'Ratio': 'ratio of the decision',
 'RPC': 'ruling by present court',
 'NONE': 'none'}

In [9]:
sample_query = dataset[0]['annotations'][0]['result'][22]['value']['text']
sample_label = dataset[0]['annotations'][0]['result'][22]['value']['labels']
sample_label

['RLC']

In [10]:
dataset[0]['annotations'][0]['result'][21]['value']['labels'][0]

'RLC'

In [11]:
prev_sent =  dataset[0]['annotations'][0]['result'][21]['value']['text']
prev_label = abbrv_label[dataset[0]['annotations'][0]['result'][21]['value']['labels'][0].strip()]
curr_sent = sample_query
print(prev_label)

ruling by lower court


In [12]:
len(dataset[0]['annotations'][0]['result'])

91

In [13]:
from tqdm import tqdm

In [14]:
def run_preds(data):
    output = ['Preamble']
    for i in tqdm(range(1, len(data))):
        
        curr_sent = dataset[0]['annotations'][0]['result'][i]['value']['text']
        prev_sent = dataset[0]['annotations'][0]['result'][i-1]['value']['text']
        if i==1:
            prev_label = 'Preamble'
        prompt = f'''You are a specialized system focused on semantic annotation of court opinion.
        RHETORICAL ROLE:
        Rhetorical roles in legal writing refer to the distinct functions or purposes that different parts of a document, such as
        a legal opinion, serve in conveying information, persuading the reader, and constructing a coherent argument. These
        roles encompass various elements like factual background, legal principles, arguments, counter arguments, and
        conclusions, each contributing to the document's overall persuasive and informative structure.
        Your task is to label each sentence in the document with one of the following predefined rhetorical roles: ['Preamble', 'Facts',
        'Ruling by Lower Court', 'Issues', 'Argument by Petitioner', 'Argument by Respondent', 'Analysis', 'Statute',
        'Precedent Relied', 'Precedent Not Relied', 'Ratio of the decision', 'Ruling by Present Court', 'NONE']
        The definition of each rhetorical role is given below:
        Preamble: A typical judgement would start with the court name, the details of parties, lawyers and judges' names, Headnotes. This section typically would end with a keyword like (JUDGEMENT or ORDER etc.) Some supreme court cases also have HEADNOTES, ACTS section. They are also part of Preamble.
        Facts: 	This refers to the chronology of events (but not judgement by lower court) that led to filing the case, and how the case evolved over time in the legal system (e.g., First Information Report at a police station, filing an appeal to the Magistrate, etc.). Depositions and proceedings of current court. Summary of lower court proceedings.
        Ruling by Lower Court: Judgments given by the lower courts (Trial Court, High Court) based on which the present appeal was made (to the Supreme Court or high court). The verdict of the lower Court, Analysis & the ratio behind the judgement by the lower Court is annotated with this label.
        Issues: Some judgements mention the key points on which the verdict needs to be delivered. Such Legal Questions Framed by the Court are ISSUES.
        Argument by Petitioner: Arguments by petitioners' lawyers. Precedent cases argued by petitioner lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
        Argument by Respondent: Arguments by respondents lawyers. Precedent cases argued by respondent lawyers fall under this but when court discusses them later then they belong to either the relied / not relied upon category.
        Analysis: Courts discussion on the evidence,facts presented,prior cases and statutes. These are views of the court. Discussions on how the law is applicable or not applicable to current case. Observations(non binding) from court. It is the parent tag for 3 tags: PRE_RLEIED, PRE_NOT_RELIED and STATUTE i.e. Every statement which belong to these 3 tags should also be marked as ANALYSIS
        Statute : Text in which the court discusses Established laws, which can come from a mixture of sources – Acts , Sections, Articles, Rules, Order, Notices, Notifications, Quotations directly from the bare act, and so on. Statute will have both the tags Analysis + Statute
        Precedent Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were relied upon by the court for final decisions. So Precedent will have both the tags Analysis + Precedent
        Precedent Not Relied: Sentences in which the court discusses prior case documents, discussions and decisions which were not relied upon by the court for final decisions. It could be due to the fact that the situation in that case is not relevant to the current case.
        Ratio of the decision: Main Reason given for the application of any legal principle to the legal issue. This is the result of the analysis by the court. This typically appears right before the final decision. This is not the same as “Ratio Decidendi” taught in the Legal Academic curriculum.
        Ruling by Present Court: Final decision + conclusion + order of the Court following from the natural / logical outcome of the rationale
        NONE: If a sentence does not belong to any of the above categories.
        
        The prediction of the current sentence highly depends on the previous sentence. Most of the time it's same but not always.
        
        
        You will be given these information to classify the rhetorical role of the current sentence: 
        1. The previous sentence.
        2. The predicted rhetorical role of the previous sentence that that you classified.

        Here is the information to help you predict the rhetorical role:
        The previous sentence: {prev_sent}
        The label of the previous sentence: {prev_label}

        Only output the rhetorical role as string.
        Now do prediction for this sentence:
        {curr_sent}
        '''

        response = llm.invoke(prompt).strip()
        prev_label = response
        output.append(response)
    
    return output
    

In [15]:
preds = run_preds(dataset[0]['annotations'][0]['result'])
len(preds)

100%|██████████| 90/90 [00:26<00:00,  3.45it/s]


91

In [16]:
import joblib
true_labels = joblib.load('./vec_prompt/true_labels')
len(true_labels)

91

In [17]:
preds[-10:], true_labels[-10:]

(['Analysis',
  'Argument by Petitioner',
  'Argument by Petitioner',
  'Analysis',
  'Ruling by Present Court',
  'Argument by Petitioner',
  'Argument by Respondent',
  'NONE',
  'NONE',
  "'NONE'"],
 ['ANALYSIS',
  'ANALYSIS',
  'RATIO',
  'ANALYSIS',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'NONE'])

In [19]:
llm_labels = []
for i, label in enumerate(preds):
    try:
        llm_labels.append(label_abbrv[label.lower().replace("'", "")])
    except:
        print(i, label.lower())


In [20]:
llm_labels[-10:], true_labels[-10:]

(['ANALYSIS',
  'ARG_PETITIONER',
  'ARG_PETITIONER',
  'ANALYSIS',
  'RPC',
  'ARG_PETITIONER',
  'ARG_RESPONDENT',
  'NONE',
  'NONE',
  'NONE'],
 ['ANALYSIS',
  'ANALYSIS',
  'RATIO',
  'ANALYSIS',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'RPC',
  'NONE'])

In [21]:
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(true_labels, llm_labels)
f1 = f1_score(true_labels, llm_labels, average="macro")

In [22]:
acc, f1

(0.27472527472527475, 0.24326777087646653)

In [26]:
llm_labels[10:20], true_labels[10:20]

(['FAC', 'FAC', 'FAC', 'FAC', 'FAC', 'FAC', 'FAC', 'FAC', 'FAC', 'FAC'],
 ['FAC',
  'ARG_RESPONDENT',
  'ARG_RESPONDENT',
  'ARG_RESPONDENT',
  'ARG_RESPONDENT',
  'FAC',
  'FAC',
  'FAC',
  'FAC',
  'FAC'])

In [23]:
from sklearn.metrics import classification_report

print(classification_report(true_labels, llm_labels))

                precision    recall  f1-score   support

      ANALYSIS       0.24      0.56      0.33         9
ARG_PETITIONER       0.14      0.18      0.16        11
ARG_RESPONDENT       0.00      0.00      0.00        17
           FAC       0.32      0.92      0.48        12
         ISSUE       0.00      0.00      0.00         0
          NONE       0.20      0.50      0.29         2
      PREAMBLE       1.00      0.75      0.86         4
    PRE_RELIED       0.50      0.04      0.07        26
         RATIO       0.00      0.00      0.00         1
           RLC       1.00      0.25      0.40         4
           RPC       1.00      0.20      0.33         5
         Ratio       0.00      0.00      0.00         0

      accuracy                           0.27        91
     macro avg       0.37      0.28      0.24        91
  weighted avg       0.37      0.27      0.22        91



/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/silly_ronny/miniconda3/envs/langchain/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [ ]:
# take top k sentences then pair them like this <sent1> <label1>....
# do sentence level classificattion with custom labels.
